In [1]:
import requests
from bs4 import BeautifulSoup
import lxml
import pandas as pd
import random
import pickle
import time
random.seed = 456

In [2]:
session = requests.Session()

In [3]:
session.headers

{'User-Agent': 'python-requests/2.25.1', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive'}

In [4]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; en-US; rv:76.0) Gecko/20100101 Firefox/76.0'}

In [5]:
session.headers.update(headers)

In [6]:
params_res = {'fillQuickSearch':'false',
              'target':'advanced',
              'expand':'dl',
              'field1':'AllField',
              'text1':"Text AND Mining OR Clustering OR Classification OR Machine AND Learning OR Data AND Mining",
              'Ppub':"[20170315 TO 20220315]",
              'pageSize':50,
              'startPage':0,
              'rel':'nofollow',
              'ContentItemType':'research-article'}

## Результаты

In [7]:
results = []

In [8]:
r = session.get('https://dl.acm.org/action/doSearch',params=params_res)

In [9]:
r.status_code

200

In [10]:
r.url

'https://dl.acm.org/action/doSearch?fillQuickSearch=false&target=advanced&expand=dl&field1=AllField&text1=Text+AND+Mining+OR+Clustering+OR+Classification+OR+Machine+AND+Learning+OR+Data+AND+Mining&Ppub=%5B20170315+TO+20220315%5D&pageSize=50&startPage=0&rel=nofollow&ContentItemType=research-article'

In [11]:
def parse_page_result(params,page):
    params['startPage'] = page
    r = session.get('https://dl.acm.org/action/doSearch',params=params)
    soup = BeautifulSoup(r.text,'lxml')
    articles_r = soup.find_all('span', class_='hlFld-Title')
    titles = []
    for article in articles_r:
        titles.append(article.text)
    links = []
    for article in articles_r:
        links.append('https://dl.acm.org'+article.findChild("a")['href'])
    return titles, links

In [12]:
def parse_info_article(url):
    r = session.get(url)
    soup = BeautifulSoup(r.text,'lxml')
    authors = []
    for author in soup.find_all('span', class_='loa__author-name'):
        authors.append(author.text)
    meta_r = soup.find('div', class_='issue-item__detail')
    source = meta_r.a['title']
    date = meta_r.find('span','epub-section__date').text.strip()
    if len(date)>15:
        date = date[12:]
    try:
        abstract_r = soup.find('div', class_='abstractSection abstractInFull').find('div', class_='abstractSection abstractInFull')
        if abstract_r is None:
            abstract_r = soup.find('div', class_='abstractSection abstractInFull')
            if abstract_r is None:
                abstract = ''
            else:
                abstract = abstract_r.text
        else:
            abstract = abstract_r.text
    except AttributeError:
        abstract = ''
    return authors, date, abstract, source

In [13]:
#pages = []
#for i in range(30):
    #if len(pages) == 0:
        #pages.append(random.randint(1,2))
    #else:
        #pages.append(pages[i-1]+random.randint(1,5))

In [14]:
pages = range(40)

In [15]:
count_results = 0
for page in pages:
    result_info = parse_page_result(params_res,page)
    count_results_page = len(result_info[0])
    for i in range(count_results_page):
        results.append({'title':'','link':'','authors':'','date':'','abstract':'', 'source':''})
    for i,j in zip(range(count_results,count_results+count_results_page),range(count_results_page)):
        results[i].update(title=result_info[0][j], link=result_info[1][j])
    count_results += count_results_page

In [17]:
with open('pre_results3.pkl','wb') as pr:
    pickle.dump(results,pr)

In [21]:
kol = 639
resp = []
for result in results[639:]:
    print(kol)
    result['authors'], result['date'], result['abstract'], result['source'] =  parse_info_article(result['link'])
    #time.sleep(0.5)
    kol+=1

639
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
659
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
680
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
715
716
717
718
719
720
721
722
723
724
725
726
727
728
729
730
731
732
733
734
735
736
737
738
739
740
741
742
743
744
745
746
747
748
749
750
751
752
753
754
755
756
757
758
759
760
761
762
763
764
765
766
767
768
769
770
771
772
773
774
775
776
777
778
779
780
781
782
783
784
785
786
787
788
789
790
791
792
793
794
795
796
797
798
799
800
801
802
803
804
805
806
807
808
809
810
811
812
813
814
815
816
817
818
819
820
821
822
823
824
825
826
827
828
829
830
831
832
833
834
835
836
837
838
839
840
841
842
843
844
845
846
847
848
849
850
851
852
853
854
855
856
857
858
859
860
861
862
863
864
865
866
867
868
869
870
871
872
873
874
875
876
877
878
879
880
881
882
883
884
885
886
887
888


In [22]:
kol

2000

In [34]:
results[1980]

{'title': 'Page-Level Main Content Extraction From Heterogeneous Webpages',
 'link': 'https://dl.acm.org/doi/10.1145/3451168',
 'authors': ['Julián Alarte', 'Josep Silva'],
 'date': 'December 2021',
 'abstract': 'The main content of a webpage is often surrounded by other boilerplate elements related to the template, such as menus, advertisements, copyright notices, and comments. For crawlers and indexers, isolating the main content from the template and other noisy information is an essential task, because processing and storing noisy information produce a waste of resources such as bandwidth, storage space, and computing time. Besides, the detection and extraction of the main content is useful in different areas, such as data mining, web summarization, and content adaptation to low resolutions. This work introduces a new technique for main content extraction. In contrast to most techniques, this technique not only extracts text, but also other types of content, such as images, and ani

In [44]:
#with open('after_results_iad3.pkl','wb') as ar:
    #pickle.dump(results,ar)

In [2]:
#with open('after_results_iad3.pkl','rb') as ar:
    #results = pickle.load(ar)

In [43]:
for result in results: # без без пробелов и \n в начале и конце
    result['abstract']=result['abstract'].strip()

In [37]:
for result in results: # без latex формул
    result['abstract']=result['abstract'].replace('(<inline-formula> <tex-math notation="LaTeX">$d$ </tex-math></inline-formula>)','')

In [ ]:
# results[1438] -- in abstract (<inline-formula> <tex-math notation="LaTeX">$d$ </tex-math></inline-formula>) (old)
# results[1448] -- abstract: '\nAs .... states.\n' (old)

In [46]:
results[1760]

{'title': 'A Fast Two-Stage Extreme Learning Machine',
 'link': 'https://dl.acm.org/doi/10.1145/3342999.3343005',
 'authors': ['Jie Lai', 'Xiaodan Wang', 'Rui Li', 'Jinghao Gu'],
 'date': 'July 2019',
 'abstract': 'In order to realize the automatic adjustment of the network structure of the extreme learning machine (ELM), inspired by the two-stage extreme learning machine (TS-ELM), a fast two-stage extreme learning machine (FTS-ELM) is proposed by making the nodes added follow the arithmetic progression and using the principal component analysis (PCA) for pruning the redundant nodes. In the growing stage of hidden nodes, the nodes are added into network according to the arithmetic progression to reduce the number of iterations. In the pruning phase, PCA is used to delete redundant nodes. The hidden nodes with low contribution rate are quickly reduced by continuously reducing the cumulative contribution rate threshold, until the error (accuracy) achieves its maximum (minimum), which mak

In [47]:
data = pd.DataFrame(results)

In [53]:
data.iloc[1968,:]

title       Network On Network for Tabular Data Classifica...
link           https://dl.acm.org/doi/10.1145/3397271.3401437
authors     [Yuanfei Luo, Hao Zhou, Wei-Wei Tu, Yuqiang Ch...
date                                                July 2020
abstract    Tabular data is the most common data format ad...
source      SIGIR '20: Proceedings of the 43rd Internation...
Name: 1968, dtype: object

In [51]:
data[data.abstract=='']

,title,link,authors,date,abstract,source
119,Evaluating Citizen Comments in Public Consulta...,https://dl.acm.org/doi/10.1145/3503823.3503902,"[Eirini Manga, Nikitas Karanikolas, Catherine ...",November 2021,,PCI 2021: 25th Pan-Hellenic Conference on Info...
525,Using Contextual Text Mining Algorithm to Anal...,https://dl.acm.org/doi/10.1145/3494885.3494919,"[C. Y. Chuang, M. C. Huang, Y. Y. Lin, Y. H. H...",October 2021,,CSSE 2021: 2021 4th International Conference o...
767,Meta-learning Fine-tuned Feature Extractor for...,https://dl.acm.org/doi/10.1145/3512353.3512355,"[Yulin Shen, Xian Shuai]",January 2022,,APIT 2022: 2022 4th Asia Pacific Information T...
991,Rock classification model based on transfer le...,https://dl.acm.org/doi/10.1145/3480571.3480595,"[Huaian Yi, Jinzhao Su, Runji Fang]",July 2021,,ICIIP 2021: 2021 6th International Conference ...
1127,Tourism English Based on Association Rules Min...,https://dl.acm.org/doi/10.1145/3469213.3472785,[Jianzhou Cui],May 2021,,ICAIIS 2021: 2021 2nd International Conference...
1227,Machine Learning-based Pipeline for Enterprise...,https://dl.acm.org/doi/10.1145/3507524.3507530,"[Guannan Wang, Wenjia Wang, Xueliang Song, Jia...",November 2021,,ICCBD 2021: 2021 4th International Conference ...
1236,Convolutional Neural Network with Transfer Lea...,https://dl.acm.org/doi/10.1145/3479645.3479687,"[Gustavo Thiodorus, Yuita Arum Sari, Novanto Y...",September 2021,,SIET '21: 6th International Conference on Sust...
1331,Effective Evaluation of Clustering Algorithms ...,https://dl.acm.org/doi/10.1145/3444884.3444886,"[Marilisa M.M. Montemurro, Gianvito G.U. Urges...",November 2020,,ICBBE '20: 2020 7th International Conference o...
1598,On Rural Power Grid Classification Method Base...,https://dl.acm.org/doi/10.1145/3510858.3510941,"[Yuanke Zhou, Fan Yang, Xiaomin Shi, Yunyun Zh...",December 2021,,ICASIT 2021: 2021 International Conference on ...
